In [1]:
#! pip install spotpy

import spotpy
import numpy as np
import pandas as pd
import json
from pathlib import Path
import bmi_cfe
import cfe
import os


In [ ]:
class Calibration:

    def __init__(self, config_dir, obs_file_path, parameter_bounds):
        self.steps = 1000
        self.config_dir = config_dir
        self.obs_file_path = obs_file_path
        self.parameter_bounds = parameter_bounds

        config_filename = 'cat-3400554.json'

        with open(Path(self.config_dir, config_filename)) as data_file:
            data_loaded = json.load(data_file)
        
        
        optguess_dict = {key: data_loaded.get(key, data_loaded["soil_params"].get(key)) for key in self.parameter_bounds.keys()}

        self.params = [
            spotpy.parameter.Uniform(name, details['lower_bound'], details['upper_bound'], optguess=optguess_dict[name])
            for name, details in self.parameter_bounds.items()
        ]

        obs_data = pd.read_csv(self.obs_file_path)
        self.observations = obs_data['Flow'].values
        self.dates = obs_data['date'].values

    def parameters(self):
        return spotpy.parameter.generate(self.params)

    def simulation(self, vector):
        
        def custom_load_forcing_file(self):
            self.forcing_data = pd.read_csv(self.forcing_file)
            
            self.forcing_data.rename(columns={"date": "time"}, inplace=True)
            
        self.generated_param = vector

        
        config_filename = './cat-3400554.json'
        with open(os.path.join(self.config_dir, config_filename), 'r') as file:
            self.cfe_cfg = json.load(file)

        self.cfe_cfg["soil_params"]['bb'] = vector[0]
        self.cfe_cfg["soil_params"]['smcmax'] = vector[1]
        self.cfe_cfg["soil_params"]['satdk'] = vector[2]
        self.cfe_cfg['slop'] = vector[3]
        self.cfe_cfg['max_gw_storage'] = vector[4]
        self.cfe_cfg['expon'] = vector[5]
        self.cfe_cfg['Cgw'] = vector[6]
        self.cfe_cfg['K_lf'] = vector[7]
        self.cfe_cfg['K_nash'] = vector[8]
        if vector[9] <= 0.5:
            self.cfe_cfg['partition_scheme'] = "Schaake"
        else:
            self.cfe_cfg['partition_scheme'] = "Xinanjiang"

       
        config_temp_filename = 'cat-3400554.json'
        with open(os.path.join(self.config_dir, config_temp_filename), 'w') as out_file:
            json.dump(self.cfe_cfg, out_file)

        self.cfemodel = bmi_cfe.BMI_CFE(cfg_file=os.path.join(self.config_dir, config_temp_filename))
        self.cfemodel.load_forcing_file = custom_load_forcing_file.__get__(self.cfemodel)


        self.cfemodel.initialize()

        with open(self.cfemodel.forcing_file, 'r') as f:
            self.df_forcing = pd.read_csv(f)

        
        self.outputs = self.cfemodel.get_output_var_names()
        self.output_lists = {output: [] for output in self.outputs}

        # Model run
        for precip, pet in zip(self.df_forcing['preci'], self.df_forcing['pet']):
            self.cfemodel.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux',
                                    precip)  # kg/m2/h = mm/h -> m/h
            self.cfemodel.set_value('water_potential_evaporation_flux', pet/3600)  # kg/m2/h = mm/h -> m/s
            self.cfemodel.update()

            for output in self.outputs:
                self.output_lists[output].append(self.cfemodel.get_value(output))

        self.cfemodel.finalize()

        self.sim_results = np.array(self.output_lists['land_surface_water__runoff_volume_flux'])  # m/h -> mm/h
        return self.sim_results

    
    def objectivefunction(self, simulation, evaluation):
        print(f"Type of simulation: {type(simulation)}")
        print(f"Type of evaluation: {type(evaluation)}")
        print(f"Simulation: {simulation}")
        print(f"Evaluation: {evaluation}")

        objectivefunction = spotpy.objectivefunctions.kge(evaluation[~np.isnan(evaluation)],simulation[~np.isnan(evaluation)])
        return objectivefunction

    def evaluation(self):
        return self.observations


with open('./CFE_parameter_bounds.json', 'r') as file:
    parameter_bounds_dict = json.load(file)

calibration_instance = Calibration(config_dir='./config/', obs_file_path='./streamflow_usgs_carmel250_2016.csv', parameter_bounds=parameter_bounds_dict)

np.random.seed(0)

sampler = spotpy.algorithms.dds(calibration_instance, dbname='raw_result_file', dbformat='ram')
N = 200
sampler.sample(N)